# [モデルの保存と復元](https://www.tensorflow.org/tutorials/keras/save_and_load)

`tf.keras`で保存と復元する方法

In [2]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.4.1


ここではMNISTデータセット（始めの1000個）を使って実験を行う。

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [4]:
# Define a simple sequential model

def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])
    
    model.compile(
        optimizer='adam',
        loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.metrics.SparseCategoricalAccuracy()]
    )
    
    return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


`tf.keras.callbacks.ModelCheckpoint`を使えば、モデルを学習中または学習後に保存することができる。

In [5]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1
)

# Train the model with the new callback
model.fit(
    train_images, 
    train_labels,  
    epochs=10,
    validation_data=(test_images, test_labels),
    callbacks=[cp_callback]
)  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/10
32/32 [==============================] - 1s 7ms/step - loss: 1.6359 - sparse_categorical_accuracy: 0.4999 - val_loss: 0.6966 - val_sparse_categorical_accuracy: 0.7840

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
32/32 [==============================] - 0s 3ms/step - loss: 0.4745 - sparse_categorical_accuracy: 0.8622 - val_loss: 0.5433 - val_sparse_categorical_accuracy: 0.8230

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
32/32 [==============================] - 0s 3ms/step - loss: 0.3059 - sparse_categorical_accuracy: 0.9080 - val_loss: 0.4933 - val_sparse_categorical_accuracy: 0.8590

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
32/32 [==============================] - 0s 3ms/step - loss: 0.2217 - sparse_categorical_accuracy: 0.9509 - val_loss: 0.4390 - val_sparse_categorical_accuracy: 0.8610

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
32/32 [==============================] - 0s 3ms/step - loss: 0.1378 - sparse_cate

In [6]:
ls {checkpoint_dir}

checkpoint                   cp.ckpt.index
cp.ckpt.data-00000-of-00001


同じモデルを重みのみを保存したファイルから復元する時は、同じ構造を持つモデルを定義し、重みを設定する。

In [7]:
# Create a basic model instance
model = create_model()

# Evaluate the untrained model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3909 - sparse_categorical_accuracy: 0.0760
Untrained model, accuracy:  7.60%


In [8]:
# Load the weights
model.load_weights(checkpoint_path)

# Re-evaluate the loaded model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4041 - sparse_categorical_accuracy: 0.8710
Restored model, accuracy: 87.10%


コールバックにはチェックポイントにユニークな名前をつけ、頻度を調節するオプションがある。

新しいモデルを学習し、チェックポイントにユニークな名前をつけ、５エポックごとにチェックポイントを作る。

In [9]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*batch_size
)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(
    train_images, 
    train_labels,
    epochs=50, 
    callbacks=[cp_callback],
    validation_data=(test_images, test_labels),
    verbose=0
)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [10]:
ls {checkpoint_dir}

checkpoint                        cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001  cp-0030.ckpt.data-00000-of-00001
cp-0000.ckpt.index                cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001  cp-0035.ckpt.data-00000-of-00001
cp-0005.ckpt.index                cp-0035.ckpt.index
cp-0010.ckpt.data-00000-of-00001  cp-0040.ckpt.data-00000-of-00001
cp-0010.ckpt.index                cp-0040.ckpt.index
cp-0015.ckpt.data-00000-of-00001  cp-0045.ckpt.data-00000-of-00001
cp-0015.ckpt.index                cp-0045.ckpt.index
cp-0020.ckpt.data-00000-of-00001  cp-0050.ckpt.data-00000-of-00001
cp-0020.ckpt.index                cp-0050.ckpt.index
cp-0025.ckpt.data-00000-of-00001


最新のモデルを評価してみる。

In [11]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [12]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4814 - sparse_categorical_accuracy: 0.8730
Restored model, accuracy: 87.30%


手動で重みを保存するには`Model.save_weights`メソッドを利用する。デフォルトでは`tf.keras`は`.ckpt`ファイルに保存する。

In [13]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4814 - sparse_categorical_accuracy: 0.8730
Restored model, accuracy: 87.30%


次は`model.save`を使って、モデル全体（構造、重み、学習時の設定）を一つのファイルやフォルダに保存する。保存には
- `SavedModel`と
- `HDF5`

の二つのフォーマットがある。

モデル全体を保存することで、TensorFlow.jsを使ってモデルを読み込みウェブブラウザ上で学習や実行を行なったり、TensorFlow Liteを使ってモデルをモバイルでバスで実行したりできる。

`SavedModel`フォーマットでモデルをシリアル化してみる。このフォーマットで保存されたモデルは`tf.keras.models.load_model`で復元でき、TensorFlow Servingと互換性がある。詳細は[SavedModel guide](https://www.tensorflow.org/guide/saved_model)

In [14]:
# Create and train a new model instance
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.6252 - sparse_categorical_accuracy: 0.5005
Epoch 2/5
32/32 [==============================] - 0s 1ms/step - loss: 0.4910 - sparse_categorical_accuracy: 0.8612
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2802 - sparse_categorical_accuracy: 0.9360
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2021 - sparse_categorical_accuracy: 0.9617
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1566 - sparse_categorical_accuracy: 0.9726
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [15]:
ls saved_model

my_model/


In [16]:
ls saved_model/my_model

assets/         saved_model.pb  variables/


`SavedModel`フォーマットで保存されたモデルの復元

In [17]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4206 - sparse_categorical_accuracy: 0.8640
Restored model, accuracy: 86.40%
(1000, 10)


次は`HDF5`フォーマットで保存と復元をしてみる。

In [19]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.5697 - sparse_categorical_accuracy: 0.5299
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4470 - sparse_categorical_accuracy: 0.8866
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.3088 - sparse_categorical_accuracy: 0.9043
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2159 - sparse_categorical_accuracy: 0.9417
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1469 - sparse_categorical_accuracy: 0.9749


In [20]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [21]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4203 - sparse_categorical_accuracy: 0.8680
Restored model, accuracy: 86.80%


Kerasは構造を調べることによってモデルに関する以下の全てのことを保存する。

- 重み
- モデルの構造
- `.compile()`メソッドで設定した学習設定
- オプティマイザとその状態（学習の再開が可能）

しかし、Kerasは`v1.x`オプティマイザ（`tf.compat.v1.train`）を保存することはできない（チェックポイントと互換性がないため）。これを使う場合は、復元した時に再度コンパイルする必要があり、オプティマイザの状態は保存されない。

カスタムオブジェクト（組み込まれたモデルや自分で作った層など）を`HDF5`フォーマットに保存する時は以下のステップが必要になる。

1. `get_config`メソッドをオブジェクトに作成する（必要ならば`from_config`クラスメソッドも作る）。
    - `get_config(self)`はオブジェクトを再作成するのに必要なパラメータの情報を持つJSONでシリアル化可能な辞書型データを返す。
    - `from_config(cls, config)`は`get_config`で返される値を使って新しいオブジェクトを作る（デフォルトでは`return cls(**config)`）。
2. モデルを復元する時に、オブジェクトを`custom_objects`という引数に渡す。引数はクラス名をPythonのクラスにマッピングする辞書型データ。（e.g., `tf.keras.model.load_model(path, custom_objects={'CustomLayer': CustomLayer})`）

詳細は[始めから層とモデルを定義する](https://www.tensorflow.org/guide/keras/custom_layers_and_models)